# Generate music, given neural net
## Piano, single track
### Based on Google 2018 Music Transformer NN

Codes recycled from:

1) Alex https://github.com/asigalov61/SuperPiano/blob/master/Super_Piano_3.ipynb

2) Damon https://github.com/gwinndr/MusicTransformer-Pytorch

3) Jason https://github.com/jason9693/midi-neural-processor

4) Mir https://github.com/mirsiadaty

Thank you :)


## 1.1: initial imports

In [70]:
import time

print(time.asctime( time.localtime( time.time() ) ))

Fri Sep  2 08:32:04 2022


## 1.2: create dir for this project

In [71]:
# params
YourHomeDir = '/home/mnt3p22/'
YourProjectSubDir = 'gen1_1'

#
!mkdir $YourHomeDir/$YourProjectSubDir
%cd $YourHomeDir/$YourProjectSubDir
!ls -ltA 

print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22/gen1_1
total 0
Fri Sep  2 08:32:14 2022


## 1.3: Check GPU and driver available

In [72]:
!nvcc --version
!nvidia-smi
print(time.asctime( time.localtime( time.time() ) ))

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Fri Sep  2 08:32:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     Off  | 00000000:3B:00.0  On |                  Off |
| 33%   45C    P0    69W / 260W |   1507MiB / 48600MiB |      5%      Default |
|                               |                      |                  N/A |
+

## 2.1: clone github lib: Jason's MIDI

In [73]:
#!git clone https://github.com/asigalov61/midi-neural-processor
!git clone https://github.com/jason9693/midi-neural-processor

print(time.asctime( time.localtime( time.time() ) ))

Cloning into 'midi-neural-processor'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 27 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (27/27), 8.56 KiB | 674.00 KiB/s, done.
Fri Sep  2 08:32:50 2022


## 2.2: clone github lib: Gwinn's NN

In [74]:
#!git clone https://github.com/asigalov61/MusicTransformer-Pytorch
!git clone https://github.com/gwinndr/MusicTransformer-Pytorch

print(time.asctime( time.localtime( time.time() ) ))

Cloning into 'MusicTransformer-Pytorch'...
remote: Enumerating objects: 346, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 346 (delta 5), reused 13 (delta 5), pack-reused 332
Receiving objects: 100% (346/346), 109.42 KiB | 3.13 MiB/s, done.
Resolving deltas: 100% (190/190), done.
Fri Sep  2 08:33:11 2022


## 2.3: QA

In [75]:
!pwd
!ls -ltA

print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22/gen1_1
total 8
drwxrwxr-x 7 mnt3p22 mnt3p22 4096 Sep  2 08:33 MusicTransformer-Pytorch
drwxrwxr-x 4 mnt3p22 mnt3p22 4096 Sep  2 08:32 midi-neural-processor
Fri Sep  2 08:33:15 2022


## 3.1: Download pre-trained NN models

In [77]:
%cd $YourHomeDir/$YourProjectSubDir

!mkdir MusicTransformer-Pytorch/rpr
!mkdir MusicTransformer-Pytorch/rpr/results


#!wget 'https://raw.githubusercontent.com/stevensiadaty/musicformer/main/SP3_20220830_0807.ipynb'
#select_model = YourHomeDir + '/' + YourProjectSubDir + '/' + "MusicTransformer-Pytorch/rpr/results/best_loss_weights.pickle" 
#github says: Yowza, that’s a big file. Try again with a file smaller than 25MB. 

%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/rpr/results/

!wget 'https://raw.githubusercontent.com/stevensiadaty/musicformer/main/best_loss_weights.pickle'


"""
%cd MusicTransformer-Pytorch/rpr/results
!wget 'https://superpiano.s3-us-west-1.amazonaws.com/SuperPiano3models.zip'
!unzip SuperPiano3models.zip
%cd MusicTransformer-Pytorch/
"""

print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22/gen1_1
mkdir: cannot create directory ‘MusicTransformer-Pytorch/rpr’: File exists
mkdir: cannot create directory ‘MusicTransformer-Pytorch/rpr/results’: File exists
/home/mnt3p22/gen1_1/MusicTransformer-Pytorch/rpr/results
--2022-09-02 15:32:17--  https://raw.githubusercontent.com/stevensiadaty/musicformer/main/best_loss_weights.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59441129 (57M) [application/octet-stream]
Saving to: ‘best_loss_weights.pickle’

best_loss_weights.p 100%[===================>]  56.69M  6.12MB/s    in 8.6s    

2022-09-02 15:32:41 (6.58 MB/s) - ‘best_loss_weights.pickle’ saved [59441129/59441129]

Fri Sep  2 15:32:41 2022


## 4.1: Download music dataset: Google Magenta MARESTRO v.2.0.0 Piano MIDI Dataset (~1300 MIDIs)

In [80]:
# capture the long output log
#%%capture capd410

#!pwd

%cd $YourHomeDir/$YourProjectSubDir

!mkdir MusicTransformer-Pytorch/dataset
%cd MusicTransformer-Pytorch/dataset/


!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip maestro-v2.0.0-midi.zip

"""
#%cd /content/MusicTransformer-Pytorch/
%cd /home/mnt3p22/MusicTransformer-Pytorch/
"""

print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22/gen1_1
mkdir: cannot create directory ‘MusicTransformer-Pytorch/dataset’: File exists
/home/mnt3p22/gen1_1/MusicTransformer-Pytorch/dataset
--2022-09-02 15:36:55--  https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.128, 172.253.115.128, 142.251.45.16, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59243107 (56M) [application/zip]
Saving to: ‘maestro-v2.0.0-midi.zip’

maestro-v2.0.0-midi 100%[===================>]  56.50M  6.71MB/s    in 8.2s    

2022-09-02 15:37:18 (6.87 MB/s) - ‘maestro-v2.0.0-midi.zip’ saved [59243107/59243107]

Archive:  maestro-v2.0.0-midi.zip
   creating: maestro-v2.0.0/
  inflating: maestro-v2.0.0/maestro-v2.0.0.csv  
   creating: maestro-v2.0.0/2008/
  inflating: maestro-v2.0.0/2008/MIDI-Unprocessed_13_R1_2008_01-04_ORIG_MID--AU

## 4.2: Prepare the directories, and the MIDI processor code

In [81]:
#%cd /content/
%cd $YourHomeDir/$YourProjectSubDir

# this is a renaming!
!mv midi-neural-processor midi_processor

# do for code preprocess_midi.py : import third_party.midi_processor.processor as midi_processor
!cp -r midi_processor/* $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/third_party/midi_processor/


#%cd /content/MusicTransformer-Pytorch/
%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/

print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22/gen1_1
/home/mnt3p22/gen1_1/MusicTransformer-Pytorch
Fri Sep  2 15:37:57 2022


## 4.3: Process the MIDI dataset

In [82]:
%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/

!python3 preprocess_midi.py $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/dataset/maestro-v2.0.0

print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22/gen1_1/MusicTransformer-Pytorch
Preprocessing midi files and saving to ./dataset/e_piano
Found 1282 pieces
Preprocessing...
50 / 1282
100 / 1282
150 / 1282
200 / 1282
250 / 1282
300 / 1282
350 / 1282
400 / 1282
450 / 1282
500 / 1282
550 / 1282
600 / 1282
650 / 1282
700 / 1282
750 / 1282
800 / 1282
850 / 1282
900 / 1282
950 / 1282
1000 / 1282
1050 / 1282
1100 / 1282
1150 / 1282
1200 / 1282
1250 / 1282
Num Train: 967
Num Val: 137
Num Test: 178
Done!

Fri Sep  2 15:43:54 2022


# 70: Generate music using the trained NN model

## 71: Assign values for parameters of music generation

In [83]:
#@title Generate, Plot, Graph, Save, Download, and Render the resulting output

number_of_tokens_to_generate = 1023 #@param {type:"slider", min:1, max:2048, step:1}
priming_sequence_length = 65 #@param {type:"slider", min:1, max:2048, step:8}
maximum_possible_output_sequence = 2048 #@param {type:"slider", min:0, max:2048, step:8}

#@param ["/content/MusicTransformer-Pytorch/rpr/results/best_acc_weights.pickle", "/content/MusicTransformer-Pytorch/rpr/results/best_loss_weights.pickle"]
select_model = YourHomeDir + '/' + YourProjectSubDir + '/' + "MusicTransformer-Pytorch/rpr/results/best_loss_weights.pickle" 

custom_MIDI = "" #@param {type:"string"}


# get to the right dir
%cd $YourHomeDir/$YourProjectSubDir/midi_processor
#
import processor
from processor import encode_midi, decode_midi

print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22/gen1_1/midi_processor
Fri Sep  2 15:45:48 2022


## 72: Execute the generation code: make music! (repeat as many as you like)

In [87]:
# auto-time-stamped subdir to save the music
DirToSaveGeneratedMusic = YourHomeDir + '/' + YourProjectSubDir + '/' + "MusicTransformer-Pytorch/generated/" + str(int(time.time()))


# get to the right dir
%cd $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch

# time keeping
startt = time.asctime( time.localtime( time.time() ) )

!python3 generate.py -output_dir $DirToSaveGeneratedMusic -model_weights=$select_model --rpr \
 -target_seq_length=$number_of_tokens_to_generate -num_prime=$priming_sequence_length \
 -max_sequence=$maximum_possible_output_sequence $custom_MIDI #

print('Successfully exported the output to output folder. To primer.mid and rand.mid')

print(startt)
print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22/gen1_1/MusicTransformer-Pytorch
midi_root: ./dataset/e_piano/
output_dir: /home/mnt3p22//gen1_1/MusicTransformer-Pytorch/generated/1662148496
primer_file: None
force_cpu: False

target_seq_length: 1023
num_prime: 65
model_weights: /home/mnt3p22//gen1_1/MusicTransformer-Pytorch/rpr/results/best_loss_weights.pickle
beam: 0

rpr: True
max_sequence: 2048
n_layers: 6
num_heads: 8
d_model: 512

dim_feedforward: 1024

Using primer index: 95 ( ./dataset/e_piano/test/MIDI-Unprocessed_17_R1_2009_01-03_ORIG_MID--AUDIO_17_R1_2009_17_R1_2009_01_WAV.midi.pickle )
RAND DIST
Generating sequence of max length: 1023
100 / 1023
150 / 1023
200 / 1023
250 / 1023
300 / 1023
350 / 1023
400 / 1023
450 / 1023
500 / 1023
550 / 1023
600 / 1023
650 / 1023
700 / 1023
750 / 1023
800 / 1023
850 / 1023
900 / 1023
950 / 1023
1000 / 1023
info removed pitch: 55
info removed pitch: 36
info removed pitch: 51
Successfully exported the output to output folder. To primer.mid and rand.mid
Fri Sep  2 15:54:56 202

## 73: List the music files generated

In [88]:
!ls -ltAR $YourHomeDir/$YourProjectSubDir/MusicTransformer-Pytorch/generated/

print(time.asctime( time.localtime( time.time() ) ))

/home/mnt3p22//gen1_1/MusicTransformer-Pytorch/generated/:
total 8
drwxrwxr-x 2 mnt3p22 mnt3p22 4096 Sep  2 15:55 1662148496
drwxrwxr-x 2 mnt3p22 mnt3p22 4096 Sep  2 15:46 1662147967

/home/mnt3p22//gen1_1/MusicTransformer-Pytorch/generated/1662148496:
total 8
-rw-rw-r-- 1 mnt3p22 mnt3p22 1522 Sep  2 15:55 rand.mid
-rw-rw-r-- 1 mnt3p22 mnt3p22  165 Sep  2 15:55 primer.mid

/home/mnt3p22//gen1_1/MusicTransformer-Pytorch/generated/1662147967:
total 8
-rw-rw-r-- 1 mnt3p22 mnt3p22 1472 Sep  2 15:46 rand.mid
-rw-rw-r-- 1 mnt3p22 mnt3p22  132 Sep  2 15:46 primer.mid
Fri Sep  2 15:55:14 2022
